In [ ]:
# import
import numpy as np
import torch
import random
# from collections import deque


In [ ]:
# initial values
action_space = [(-3, True), (-1, True), (0, True), (1, True), (2, True), (-3, False) (-1, False), (0, False),(1, False), (2, False)] # discrete action space; (acceleration, lane change)
n_actions = 8 # number of actions
n_features = 4  # number of features in state [current, prev_1, prev_2, prev_3]
len = 1000 # length of a lane
num_lane = 2 # number of lanes
Road = [i for i in range(num_lane)]

init_v_auto = 10 # initial velocity of agent
init_v_v1 = [5, 8] # initial velocity of vehicle 1; randomly chosen
init_v_v2 = [10, 15] # initial velocity of vehicle 2; randomly chosen
init_v_v3 = [8, 13] # initial velocity of vehicle 3; randomly chosen
init_gap_AuV1 = [30, 50] # initial gap between agent and vehicle 1; randomly chosen
init_gap_AuV2 = [60, 200] # initial gap between agent and vehicle 2; randomly chosen
init_gap_V2V3 = [40, 340] # initial gap between vehicle 2 and vehicle 3; randomly chosen

time_step = 0.5 # time step

T = 40 # Total simulation time for MCTS??

In [ ]:
class Feature(object): # abstract feature of environment
    def __init__(self, s_e, s_ov):
        self.s_e = s_e # (velocity, lane ID) of autonomous vehicle
        self.s_ov = s_ov # [(relative position, velocity) * 6] of surronding vehicles   ----- why 6??
    

class State(object):
    def __init__(self, feat_cur, feat_prev1, feat_prev2, feat_prev3):
        self.features = [feat_cur, feat_prev1, feat_prev2, feat_prev3]

In [ ]:
# Reward functions for MDP
# Reward function for DRQN
class Reward_RL(object):
    def __init__(self):
        self.reward_effi = 0
        self.reward_safety = 0
        self.reward_ter = 0

    def cal_reward(self, alpha1, alpha2, s_t):
        # Efficiency reward
        self.reward_effi = alpha1 * (s_t[0][0][0] - 15)
        # Safety reward
        self.reward_safety = alpha2 * min(0, ttc - 2.7) + c_op*s_ov
        # Terminal reward
        if terminal == "overtake":
            self.reward_ter = 10
        elif terminal == "collision":
            self.reward_ter = -10
        else:
            self.reward_ter = 0

        
# Reward function for safe exploration
class Reward_MCTS(object):
    def __init__(self, ):
        self.r_mcts = -0.5
        
    def cal_reward(self, risk, a_rl, a_se):
        if risk == 0  or a_rl == a_se:
            self.r_mcts = 0
        else:
            self.r_mcts = -0.5

        return self.r_mcts

In [ ]:
# Scenario: Two-lane overtaking
class TwoLane(object):
    def __init__(self, action_space, n_actions, n_features, len, num_lane): # initialization
        self.action_space = action_space
        self.n_actions = n_actions
        self.n_features = n_features
        self.len = len
        self.num_lane = num_lane


        # Initialize state
        s_e = (init_v_auto, 0)
        s_ov = []
        veh1 = (random.uniform(init_gap_AuV1[0], init_gap_AuV1[1]), random.uniform(init_v_v1[0], init_v_v1[1]))

        rand = random.uniform(init_gap_AuV2[0], init_gap_AuV2[1])
        if rand < 150:
            veh2 = (rand, random.uniform(init_v_v2[0], init_v_v2[1]))
        else:
            veh2 = (150, -1)

        rand = veh2[0]+random.uniform(init_gap_V2V3[0], init_gap_V2V3[1])
        if rand < 150:
            veh3 = (rand, veh2[1]+random.uniform(init_v_v3[0], init_v_v3[1]))
        else:
            veh3 = (150, -1)

        s_ov.append(veh1)
        s_ov.append(veh2)
        s_ov.append(veh3)
                

        self.s_t = [Feature(s_e, s_ov)]

        s_e_i = (0, 0)
        s_ov_i = [(150, -1), (150, -1), (150, -1)]

        self.s_t.append([Feature(s_e_i, s_ov_i) * 3]) # [feat_cur, feat_initial, feat_initial, feat_initial]
        

    def reset(self): # reset for new episode
        s_e = (init_v_auto, 0)
        s_ov = []
        veh1 = (random.uniform(init_gap_AuV1[0], init_gap_AuV1[1]), random.uniform(init_v_v1[0], init_v_v1[1]))

        rand = random.uniform(init_gap_AuV2[0], init_gap_AuV2[1])
        if rand < 150:
            veh2 = (rand, random.uniform(init_v_v2[0], init_v_v2[1]))
        else:
            veh2 = (150, -1)

        rand = veh2[0]+random.uniform(init_gap_V2V3[0], init_gap_V2V3[1])
        if rand < 150:
            veh3 = (rand, veh2[1]+random.uniform(init_v_v3[0], init_v_v3[1]))
        else:
            veh3 = (150, -1)

        s_ov.append(veh1)
        s_ov.append(veh2)
        s_ov.append(veh3)
                

        self.s_t = [Feature(s_e, s_ov)]

        s_e_i = (0, 0)
        s_ov_i = [(150, -1), (150, -1), (150, -1)]

        self.s_t.append([Feature(s_e_i, s_ov_i) * 3])


    def step(self, action): # state transition

        # Get next feature
        # Check lane change
        if action[1]:
            lane = self.s_t[0].s_e[1] or action[1] # Works only for two lane [0, 1]
          
        # Calculate new velocity of autonomous vehicle
        new_v_auto = self.s_t[0].s_e[0] + action[0] * time_step

        new_s_e = (new_v_auto, lane)
        # Calculate new position and velocity for other vehicles
        new_s_ov = []
        
        veh1 = (random.uniform(init_gap_AuV1[0], init_gap_AuV1[1]), random.uniform(init_v_v1[0], init_v_v1[1]))
        # how to calculate veh2 and veh 3? -> Intelligent Driver Model

        
        
            

In [ ]:
# Safe exploration
# Risk state estimation
# Safe policy search

In [ ]:
# DRQN from now on